In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

df = pd.read_csv('C:/Users/JSWIFT1/OneDrive - KNEX/Desktop/New folder/New folder/machine learning/wine.csv')

# Remove rows with outliers
z_scores = np.abs((df.select_dtypes(include=[np.number]) - df.select_dtypes(include=[np.number]).mean()) / df.select_dtypes(include=[np.number]).std())
outliers = df[(z_scores > 3).any(axis=1)]
df = df.drop(outliers.index)

# Remove duplicate rows
df = df.drop_duplicates()

# Identify columns with few distinct values
distinct_counts = df.nunique()
columns_with_few_distinct = distinct_counts[distinct_counts <= 2].index.tolist()

# Remove columns with few distinct values
df = df.drop(columns_with_few_distinct, axis=1)

# Remove columns with missing values 
columns_with_missing_values = df.columns[df.isnull().any()].tolist()
df = df.drop(columns=columns_with_missing_values)

# Check if 'y' column exists
if 'y' not in df.columns:
    print("Error: 'y' column not found in the dataset.")
else:
    # Convert 'y' column to categorical type
    df['y'] = df['y'].astype('category')

    # Divide the cleaned dataset into features (X) and target variable (y)
    X = df.drop(columns=['y'])
    y = df['y']

    # Convert categorical variable to numerical labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    # missing values
    imputer = SimpleImputer(strategy='mean')
    X = imputer.fit_transform(X)

    #1 Divide the cleaned dataset into training and test subsets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    #2 Use GridSearchCV to auto-tune at least three classification algorithms using the training dataset
    classifiers = [
        ('Random Forest', RandomForestClassifier(), {'n_estimators': [50, 100, 200]}),
        ('SVM', SVC(), {'C': [1, 10, 100], 'gamma': [0.1, 1, 10]}),
        ('KNN', KNeighborsClassifier(), {'n_neighbors': [3, 5, 7]})
    ]

    best_model = None
    best_score = 0

    for name, classifier, param_grid in classifiers:
        grid_search = GridSearchCV(classifier, param_grid, cv=5)
        grid_search.fit(X_train, y_train)

        if grid_search.best_score_ > best_score:
            best_score = grid_search.best_score_
            best_model = grid_search.best_estimator_
            best_model_name = name

    print(f"Best Model: {best_model_name}")
    print(f"Best Model Parameters: {best_model.get_params()}")

    #3 Calculate the cross-validation mean and standard deviation of the best model
    cross_val_scores = cross_val_score(best_model, X_train, y_train, cv=5)
    cross_val_mean = cross_val_scores.mean()
    cross_val_std = cross_val_scores.std()

    print(f"Cross-Validation Mean: {cross_val_mean}")
    print(f"Cross-Validation Standard Deviation: {cross_val_std}")

    #4 Fit the model to the training dataset
    best_model.fit(X_train, y_train)

    #5 Calculate the scores of the training and test datasets
    train_score = best_model.score(X_train, y_train)
    test_score = best_model.score(X_test, y_test)

    print(f"Training Score: {train_score}")
    print(f"Test Score: {test_score}")

C:\Users\JSWIFT1\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\JSWIFT1\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `kee

Best Model: Random Forest
Best Model Parameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
Cross-Validation Mean: 0.9703703703703702
Cross-Validation Standard Deviation: 0.014814814814814836
Training Score: 1.0
Test Score: 1.0
